In [72]:
import pandas as pd
from datetime import date

In [73]:
food_df = pd.read_csv('myfitnesspal/mfp-extract-2021.csv',index_col=0)
food_df.head()

,food,calories_kcal,carbs_g,fat_g,protein_g,sugar_g,fiber_g,goal_food,goal_calories_kcal,goal_carbs_g,goal_fat_g,goal_protein_g,goal_sugar_g,goal_fiber_g,date
0.0,"Eggs - White only, raw, 3 large",51,1 6,0 3,11 91,1,0,Your Daily Goal,2000,225 45,56 25,150 30,99,38,2021-01-01
1.0,"Eggs - Whole, raw, 3 medium",194,1 2,13 63,17 35,1,0,Your Daily Goal,2000,225 45,56 25,150 30,99,38,2021-01-01
2.0,"Apples - Raw, with skin, 250 g",130,35 96,0 3,1 1,26,6,Your Daily Goal,2000,225 45,56 25,150 30,99,38,2021-01-01
3.0,"The Cheese Emporium - Mature Lighter Cheese, 30 g",96,1 3,7 63,8 34,1,1,Your Daily Goal,2000,225 45,56 25,150 30,99,38,2021-01-01
4.0,"Hovis - 50/50 Bread, 3 slice",273,48 75,2 6,12 19,4,6,Your Daily Goal,2000,225 45,56 25,150 30,99,38,2021-01-01


In [74]:
#reset index so each line has unique index (i.e. indices not restarted from 0 for each new day)
food_df.reset_index(drop=True, inplace=True)

In [75]:
#drop all rows containing column headers are repeated (i.e. where dataframe is appended)
food_df.drop(food_df[food_df['food']=='food'].index, inplace=True)

In [76]:
#check column data types
food_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3191 entries, 0 to 3554
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   food                3191 non-null   object
 1   calories_kcal       3191 non-null   object
 2   carbs_g             3191 non-null   object
 3   fat_g               3191 non-null   object
 4   protein_g           3191 non-null   object
 5   sugar_g             3191 non-null   object
 6   fiber_g             3191 non-null   object
 7   goal_food           3191 non-null   object
 8   goal_calories_kcal  3191 non-null   object
 9   goal_carbs_g        3191 non-null   object
 10  goal_fat_g          3191 non-null   object
 11  goal_protein_g      3191 non-null   object
 12  goal_sugar_g        3191 non-null   object
 13  goal_fiber_g        3191 non-null   object
 14  date                3191 non-null   object
dtypes: object(15)
memory usage: 398.9+ KB


all columns should be numeric except for 'date' (datetime) and 'food' (str) - but the columns for macronutrients (fats, carbs and protein) have two numbers separated by a space so it needs to be sorted before we try convert to a numeric datatype

In [77]:
macro_cols = ['carbs_g', 'fat_g', 'protein_g', 'goal_carbs_g', 'goal_fat_g', 'goal_protein_g']
food_df[macro_cols].head()

,carbs_g,fat_g,protein_g,goal_carbs_g,goal_fat_g,goal_protein_g
0,1 6,0 3,11 91,225 45,56 25,150 30
1,1 2,13 63,17 35,225 45,56 25,150 30
2,35 96,0 3,1 1,225 45,56 25,150 30
3,1 3,7 63,8 34,225 45,56 25,150 30
4,48 75,2 6,12 19,225 45,56 25,150 30


The data was scraped from the myfitnesspal webpages (see scrape_mfp.py) so if we inspect the page source - we can see the macro percentage is stored in the same cell as the value with a css display property of none (which just hides the value from the user)

The function used to read html tables into a dataframe: pd.read_html() ignores the span tags and renders all the text contents of the html cell into a single dataframe cell

![mfp source code detective work]('./images/macro-perc-gotcha.png')

![display is set to none via css]('./images/hidden-macro-perc.png')

Since we're only interested in the macro value we can split the string on the whitespace (default) and take the first value

In [78]:
food_df[macro_cols] = food_df[macro_cols].applymap(lambda x:x.split("  ")[0])

#convert datatypes
numeric_cols =[col for col in food_df.columns if '_g' in col or 'calories' in col]
food_df[numeric_cols] = food_df[numeric_cols].apply(pd.to_numeric)
food_df['date'] = pd.to_datetime(food_df['date'])

#check datatypes again
food_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3191 entries, 0 to 3554
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   food                3191 non-null   object        
 1   calories_kcal       3191 non-null   int64         
 2   carbs_g             3191 non-null   int64         
 3   fat_g               3191 non-null   int64         
 4   protein_g           3191 non-null   int64         
 5   sugar_g             3191 non-null   int64         
 6   fiber_g             3191 non-null   int64         
 7   goal_food           3191 non-null   object        
 8   goal_calories_kcal  3191 non-null   int64         
 9   goal_carbs_g        3191 non-null   int64         
 10  goal_fat_g          3191 non-null   int64         
 11  goal_protein_g      3191 non-null   int64         
 12  goal_sugar_g        3191 non-null   int64         
 13  goal_fiber_g        3191 non-null   int64       

The 'food' column contains names of food items but it also has the quantities - can we extract more data out of this?

In [113]:
# can we use the comma as a consistent separator to return [food_name, food_qty]?
food_list_df =  food_df['food'].agg({'list':lambda x:x.split(','), 'list_size':lambda x:len(x.split(','))}).unstack().transpose()

food_list_df['list_size'].value_counts(normalize=True)

2    0.861799
3    0.134754
6    0.003134
5    0.000313
Name: list_size, dtype: float64

~86% of data can be cleaned this way but there are some exceptions - we'll use the last element of the split list which should capture most of the serving sizes even for the rows with more than one comma

In [127]:
# top offenders
food_list_df[food_list_df['list_size']>2]


# we can use the last element of the list but there'll be some exception that we are going just accept won't be clean
print("example of data that won't be cleaned accurately\n'",
",".join(food_list_df[food_list_df['list_size']>2].loc[12, 'list']), "' - the quantity will be parsed as '00 gram'")


example of data that won't be cleaned accurately
' Breyers - Creamy chocolate, 1 container (264,00 gram) ' - the quantity will be parsed as '00 gram'


In [133]:
food_df['food_name'] = food_df['food'].apply(lambda x:"".join(x.split(',')[:-1]))
food_df['food_qty'] = food_df['food'].apply(lambda x:"".join(x.split(',')[-1]))
food_df.drop('food', axis=1, inplace=True)
food_df.head()

Let's try query some data!

e.g. how many calories did I eat on new years day in 2021?

In [138]:
new_year_food = food_df[food_df['date'].apply(lambda x:x.date()==date(2021,1,1))]
print(f"I ate {new_year_food['calories_kcal'].sum()} calories (target: {int(new_year_food['goal_calories_kcal'][0])})")

I ate 2386 calories (target: 2000)
